# Déploiement d'un modèle de régression à l'aide de MLflow / Ray / Cortex

## Déroulé du webinaire
Dans cet exercice, nous allons entraîner un modèle de régression afin de prédire le prix médian des maisons de la Californie. Ensuite, nous déploierons ce modèle à travers une API REST en utilisant trois librairies dédiées : MLflow, Ray et Cortex. Ces librairies permettent toutes de déployer des modèles de machine learning, c'est en tout cas une de leurs fonctionnalités ! pourquoi toutes les utiliser alors ? nous allons les comparer !
Vous pourrez tester vos prédictions avec une interface Streamlit qui est déjà implémentée pour vous ;)

Pour résumer, au programme :
1. Récupération des données, recherche des hyperparamètres, construction d'un pipeline.
2. Déploiement de l'API REST avec MLflow.
3. Interaction du tableau de bord Streamlit avec l'API MLflow.
4. Déploiement de l'API REST avec Ray et Cortex.

Du côté des librairies utiles pour le projet : scikit-learn, mlflow, ray, cortex, streamlit.

### Nota Bene
Vous avez peut-être déjà entendu parler de Flask qui est une librairie Python pour construire des APIs, c'est d'ailleurs le [sujet d'un cours](https://openclassrooms.com/fr/courses/4525361-realisez-un-dashboard-avec-vos-donnees/5774811-creez-une-api-avec-flask) sur OpenClassrooms !

Dans cet exercice, nous n'utiliserons pas Flask, l'objectif est au contraire de vous faire votre point de vue sur des outils différents, plus spécifiques, conçus uniquement pour répondre aux problématiques propres au machine learning.


## Introduction au déploiement d'un modèle
Une fois le travail de modélisation terminé, il est temps de mettre votre travail à disposition de vos utilisateurs.

Nous allons pour cela créer une API REST qui donnera accès à votre modèle, voici le déroulé de son utilisation en pratique :
1. **Envoi par l'utilisateur** d'une requête (contenant les données d'entrée du modèle) au service web en charge de la prédiction.
2. **Réception par le service** de la requête, le modèle pré-traite et prédit un résultat.
3. **Envoi par le service** de la prédiction en réponse à la requête utilisateur originale.
4. **Réception par l'utilisateur** de la prédiction et affichage à l'écran.

Si les termes que nous avons utilisé ne vous semblent pas clairs, c'est le moment de vous former sur ce sujet important qu'est l'utilisation d'une API REST :
- https://openclassrooms.com/fr/courses/4525361-realisez-un-dashboard-avec-vos-donnees/5774786-apprehendez-le-fonctionnement-dun-serveur-web
- https://openclassrooms.com/fr/courses/6573181-adoptez-les-api-rest-pour-vos-projets-web
- https://practicalprogramming.fr/api-rest/



## 1 - Entraînez un modèle de régression (45 minutes)
### Présentation du jeu de données

Le jeu de données California Housing synthétise la situation immobilière de la Californie par quartier à l'aide de 8 variables :
- `MedInc` revenu médian dans le secteur (en 10K $)
- `HouseAge` âge médian des maisons dans le secteur
- `AveRooms` nombre moyen de pièces
- `AveBedrms` nombre moyen de chambres
- `Population` taille de la population dans le secteur
- `AveOccup` occupation moyenne de la maison
- `Latitude` latitude du secteur
- `Longitude` longitude du secteur

Avec ces variables prédictives, l'objectif est d'estimer le prix médian des maisons pour un secteur donné.

In [1]:
from sklearn import datasets, preprocessing, model_selection, ensemble, pipeline
from sklearn.experimental import enable_hist_gradient_boosting

### Téléchargement du jeu de données
<img src='./images/logo_oc.png' width=15px /> [Suivre le lien](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing) pour télécharger les données. Extrayez uniquement du jeu de données :
- Les variables prédictives dans la matrice `X`.
- La variable cible dans le vecteur `y`.

In [2]:
X, y = datasets.fetch_california_housing(return_X_y=True)

<img src='./images/logo_oc.png' width=15px /> Vérifiez la taille de la matrice `X` et du vecteur `y`.

In [3]:
X.shape, y.shape

((20640, 8), (20640,))

### Séparation des données

Afin de mesurer les performances de notre modèle sur des données de test, nous allons séparer les données en deux bases : apprentissage et test.

[Cours pour bien comprendre comment évaluer un modèle](https://openclassrooms.com/fr/courses/4297211-evaluez-les-performances-dun-modele-de-machine-learning)

<img src='./images/logo_oc.png' width=15px /> Séparez les deux variables (`X` et `y`) en deux bases, retenez 80% des données pour l'apprentissage.

In [4]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.8)

<img src='./images/logo_oc.png' width=15px /> Vérifiez la taille des matrices `X_train` et `X_test`.

In [5]:
X_train.shape, X_test.shape

((16512, 8), (4128, 8))

### Pré-traitement des données

Nous allons maintenant centrer et réduire nos variables prédictives, c'est la [standardisation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Attention à la fuite de données, la base de test doit être normalisée avec les paramètres appris sur la base d'apprentissage. 

<img src='./images/logo_oc.png' width=15px /> Effectuez la standardisation des variables `X_train` et `X_test`.

In [6]:
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Recherche des hyperparamètres

Pour modéliser la relation qu'il existe entre les variables prédictives `X_train` et la cible `y_train`, nous allons utiliser un modèle ensembliste basé sur le boosting : [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor).

**Le choix du modèle n'est pas très important pour cet exercice**, si jamais vous avez un blocage, utilisez un modèle de régression linéaire ou un arbre de décision.

Pour le modèle `HistGradientBoostingRegressor` nous allons rechercher les hyperparamètres les plus adaptés afin d'obtenir les meilleures performances, utilisez pour cela la classe `GridSearchCV`. Les hyper-paramètres importants sont : `learning_rate`, `max_depth`, `min_samples_leaf`, `max_iter`. Référez vous à la documentation pour bien les comprendre !

**Indices** :
- learning_rate entre 1e-1 et 3e-1
- max_depth entre 2 et 5
- min_samples_leaf entre 30 et 32
- max_iter entre 100 et 150

<img src='./images/logo_oc.png' width=15px /> Effectuez la recherche des hyperparamètres par validation-croisée.

In [7]:
regressor = ensemble.HistGradientBoostingRegressor()
params = {'learning_rate': [1e-1, 2e-1, 3e-1],
          'max_depth': [2, 4, 5],
          'min_samples_leaf': [30, 31, 32],
          'max_iter': [100, 150]
          }
gsv = model_selection.GridSearchCV(regressor, params, cv=5)
gsv.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=HistGradientBoostingRegressor(),
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': [2, 4, 5], 'max_iter': [100, 150],
                         'min_samples_leaf': [30, 31, 32]})

<img src='./images/logo_oc.png' width=15px /> Affichez le meilleur score obtenu ainsi que les meilleurs hyperparamètres.

In [8]:
gsv.best_score_, gsv.best_params_

(0.8362064442371151,
 {'learning_rate': 0.2,
  'max_depth': 5,
  'max_iter': 150,
  'min_samples_leaf': 32})

<img src='./images/logo_oc.png' width=15px /> Affichez le coefficient de détermination moyen sur la base de test (faites un scoring de la base de test). *Pas de problème si votre score est moins bon ! ce n'est pas l'objectif de l'exercice.*

In [9]:
gsv.best_estimator_.score(X_test_scaled, y_test)

0.8294976590349341

### [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) de prédiction

Nous souhaitons maintenant réunir toutes les étapes nécessaires à la prédiction dans des conditions réelles, pour cela nous devons regrouper la standardisation et la régression dans un seul outil, c'est le rôle d'un pipeline.

<img src='./images/logo_oc.png' width=15px /> Construisez un [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) de prédiction intégrant l'étape de [pré-traitement](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) et la [régression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html). N'oubliez pas d'utiliser les hyperparamètres que vous avez trouvé à l'étape précédente pour configurer votre modèle.

In [10]:
pipeline = pipeline.Pipeline([('scaler', preprocessing.StandardScaler()), 
                              ('regressor', ensemble.HistGradientBoostingRegressor(**gsv.best_params_))])

<img src='./images/logo_oc.png' width=15px /> Lancez l'apprentissage du pipeline sur les données d'apprentissage

In [11]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 HistGradientBoostingRegressor(learning_rate=0.2, max_depth=5,
                                               max_iter=150,
                                               min_samples_leaf=32))])

<img src='./images/logo_oc.png' width=15px /> Calculez les performances (coefficient de détermination) sur la base de test, le résultat devrait être proche du précédent.

In [12]:
pipeline.score(X_test, y_test)

0.8311388338846891

Le pipeline regroupe maintenant toutes les étapes nécessaires pour prédire sur des données, nous allons maintenant le déployer.

<img src='./images/logo_oc.png' width=15px /> Sérialiser le pipeline à l'aide de [joblib](https://scikit-learn.org/stable/modules/model_persistence.html), sous le nom de fichier `pipeline_housing.joblib`.

In [17]:
import joblib

In [19]:
joblib.dump(pipeline, 'pipeline_housing.joblib')

['pipeline_housing.joblib']

## Pour la suite des exercices

À partir de maintenant, **il est vivement conseillé d'utiliser le terminal** de votre système d'exploitation pour installer les librairies et pour lancer les déploiements le moment venu. 

En effet, un notebook jupyter n'est pas un environnement adapté pour ce type de travail, qui est à la frontière de l'administration système et du développement, c'est le [MLOps](https://www.google.com/search?q=mlops) !

## 2 - Déploiement d'un modèle sklearn avec MLflow (40 minutes)

### Installation avec Pip

<img src='./images/logo_oc.png' width=15px /> Exécutez la commande suivante dans le terminal associé à votre environnement Python `pip install mlflow`

### Présentation de MLflow

> MLflow is a platform to streamline machine learning development, including tracking experiments, packaging code into reproducible runs, and sharing and **deploying models**. MLflow offers a set of lightweight APIs that can be used with any existing machine learning application or library (TensorFlow, PyTorch, XGBoost, etc), wherever you currently run ML code (e.g. in notebooks, standalone applications or the cloud). MLflow's current components are:
>- MLflow Tracking: An API to log parameters, code, and results in machine learning experiments and compare them using an interactive UI.
>- MLflow Projects: A code packaging format for reproducible runs using Conda and Docker, so you can share your ML code with others.
>- **MLflow Models: A model packaging format and tools that let you easily deploy the same model (from any ML library) to batch and real-time scoring on platforms >such as Docker, Apache Spark, Azure ML and AWS SageMaker.**
>- MLflow Model Registry: A centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of MLflow Models.

[Source](https://github.com/mlflow/mlflow)

--------

MLflow est une librairie dédiée à la gestion du cycle de vie d'un projet de machine learning, à savoir :
- le suivi des résultats liés aux expériences (MLflow Tracking)
- garantir la reproducibilité des expériences et le partage de code (MLflow Projects)
- la gestion des modèles sous un format normalisé afin de simplifier les déploiements locaux ou cloud (MLflow Models)
- la centralisation et le versionnage les modèles (MLflow Model Registry)

**On s'intéresse dans le cadre de l'exercice à MLflow Models uniquement**, vous pouvez explorer les autres fonctionnalités bien entendu, cependant ce n'est pas l'objectif de cet exercice.

-------

### Format de stockage d'un modèle

Un modèle MLflow est un répertoire contenant une liste de fichiers, dont un nommé MLmodel qui liste les différentes flavors (terme MLflow) dans lesquelles le modèle est utilisable.

Les flavors sont une façon pratique de définir comment exécuter un modèle, cette convention facilite le déploiement car la flavor standardise la façon de prédire à partir d'un modèle. MLflow définit par exemple des flavors pour une fonction python, sklearn, tensorflow, xgboost.

Dans le répertoire de sauvegarde du modèle MLflow, le fichier MLmodel contient l'ensemble des flavors disponibles, par exemple :
```yaml
time_created: 2018-05-25T17:28:53.35

flavors:
  sklearn:
    sklearn_version: 0.19.1
    pickled_model: model.pkl
  python_function:
    loader_module: mlflow.sklearn
```

Tous les outils supportant les flavors `python_function` ou `sklearn` pourront utiliser ce modèle, par exemple pour un déploiement avec la commande suivante.

```python
mlflow models serve -m my_model
```

D'autres outils cloud comme AWS SageMaker ou Azure ML peuvent utiliser ces flavors.

[Documentation pour approfondir](https://mlflow.org/docs/latest/models.html#storage-format)

-----------

### Signature d'un modèle

Afin de garantir que les données d'entrée d'un modèle sont conformes à ce qui est attendu, les modèles MLflow peuvent inclure des metadatas décrivant les entrées et sorties :
- Model Signature - description des entrées et sorties du modèle
- Model Input Example - exemple d'une entrée valide

La signature du modèle permet de renseigner le nom des colonnes et leurs types afin de vérifier si ils sont similaires lors de la prédiction.

[Documentation](https://mlflow.org/docs/latest/models.html#model-signature-and-input-example)

In [13]:
from mlflow.models.signature import infer_signature

<img src='./images/logo_oc.png' width=15px />  Utiliser la fonction `infer_signature` afin d'extraire la signature à partir des données d'entrée et de sortie.

In [14]:
signature = infer_signature(X_train, y_train)

------

### Sauvegarde du modèle sklearn

In [15]:
import mlflow.sklearn

<img src='./images/logo_oc.png' width=15px />  Sauvegarder le pipeline à l'aide de la fonction [save_model](https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.save_model) en n'oubliant pas de préciser la signature.

In [16]:
mlflow.sklearn.save_model(pipeline, 'mlflow_model', signature=signature)

<img src='./images/logo_oc.png' width=15px />  Vérifier qu'un répertoire correspondant au modèle vient bien d'être généré. Ce dernier doit contenir trois fichiers.

 ------------

### Déploiement d'une API REST

Les commandes suivantes sont à lancer **depuis le terminal** associé à votre environnement Python.

<img src='./images/logo_oc.png' width=15px />  À partir du modèle MLflow, vous n'êtes plus qu'à une commande de lancer un serveur pour votre API REST ([documentation](https://mlflow.org/docs/latest/models.html#deploy-mlflow-models)). À vous de jouer ! [(en cas de blocage)](https://mlflow.org/docs/latest/cli.html#mlflow-models-serve)

<img src='./images/logo_oc.png' width=15px />  Écrivez la requête curl pour envoyer une requête. Faites attention aux guillemets :)

Si votre requête ne fonctionne pas, **ne bloquez pas**, vous pourrez tout de même tester votre API avec le dashboard qui est déjà configuré pour envoyer une requête.

 --------

### Conclusion et perspectives de MLflow

L'utilisation de MLflow pour le déploiement offre des options intéressantes, nous avons vu comment mettre en place une API REST, nous aurions également pu générer une image Docker pour la déployer sur un service cloud comme [Azure ML](https://docs.microsoft.com/fr-fr/azure/databricks/_static/notebooks/mlflow/mlflow-quick-start-deployment-azure.html). De plus la fonctionnalité MLflow Models n'est qu'une des quatre grandes qui forment MLflow, le potentiel n'est donc pas complètement exploré.

### Ressources

- [Documentation MLflow](https://www.mlflow.org/docs/latest/index.html)
- [Déploiement avec Azure ML](https://docs.microsoft.com/fr-fr/azure/databricks/_static/notebooks/mlflow/mlflow-end-to-end-example-azure.html)
- [Documentation MLflow Microsoft](https://docs.microsoft.com/fr-fr/azure/databricks/applications/mlflow/)

## 3 - Configuration du tableau de bord Streamlit (10 minutes)

Nous allons maintenant connecter un tableau de bord qui nous a été fourni par une autre équipe, à notre API MLflow. Le tableau de bord est implémenté avec la librairie [Streamlit](https://www.streamlit.io/), elle est très simple d'utilisation, en quelques minutes on peut mettre en place des champs de saisie et des affichages, le tout en Python.

<img src='./images/logo_oc.png' width=15px />  Regardez dans le dossier racine, il contient le fichier `dashboard.py` qui contient le descriptif de l'interface et une fonction permettant d'envoyer une requête.

<img src='./images/logo_oc.png' width=15px />  Affectez l'adresse du serveur de l'API à la variable `MLFLOW_URI` dans la fonction `main`.

<img src='./images/logo_oc.png' width=15px />  Installez Streamlit si ce n'est pas déjà fait :

`pip install streamlit`

<img src='./images/logo_oc.png' width=15px />  Avec la console, depuis le répertoire racine, lancer Streamlit avec la commande suivante :

`streamlit run dashboard.py`

Vous devriez voir apparaitre un nouvel onglet dans votre navigateur, c'est le tableau de bord qui s'affiche !

<img src='./images/streamlit.png' width=500px />

<img src='./images/logo_oc.png' width=15px />  Lancez une prédiction en appuyant sur le bouton en bas de page, si vous avez bien configuré le serveur de l'API et que l'adresse est correcte, un résultat sera affiché.

## 4 - Déploiement d'un pipeline sklearn avec Ray Serve (40 minutes)

### Installation avec Pip

<img src='./images/logo_oc.png' width=15px /> Executez les commandes suivantes depuis votre terminal :

```bash
pip install -U ray
pip install "ray[serve]"
```

### Présentation de Ray Serve
Ray est une librairie python qui permet de construire des applications distribuées (plusieurs processus/machines).

Voici la liste des fonctionnalités de Ray :
- Propose des primitives simples pour construire et exécuter des applications distribuées.
- Permet la parallélisation de code utilisateur, avec peu voir aucun changement de code.
- Intégration avec de nombreuses applications et librairies de l'écosystème de la data (Dask, Horovod, Modin).

Pour vous rassurer, nous n'allons pas construire d'application distribuée, par contre il faut comprendre que Ray est un outil qui est utilisé par d'autres librairies comme socle pour la distribution des calculs, dans le but d'accélérer l'obtention d'un résultat.

Plusieurs librairies de machine learning sont construites en s'appuyant sur Ray :
- Tune: recherche d'hyperparamètres scalable
- RLlib: apprentissage par renforcement scalable
- Ray Serve: déploiement d'API scalable
- RaySGD: wrappers pour apprentissage distribué

[source](https://docs.ray.io/en/master/ray-overview/index.html)

**Nous allons justement utiliser Ray Serve afin de déployer notre pipeline**, nous réaliserons cela en écrivant exclusivement du Python. Ray Serve n'est pas dépendant d'autres frameworks ou librairies, vous pourriez donc en théorie déployer n'importe quel type de service écrit en Python (modèle ML ou autre) !

1. Il est maintenant temps que vous commenciez à vous renseigner sur [son fonctionnement](https://docs.ray.io/en/master/serve/#ray-serve-in-90-seconds).

2. Vous devriez maintenant avoir compris les **deux notions de `endpoint` et `backend`**, continuez dans [cette direction](https://docs.ray.io/en/master/serve/key-concepts.html).

3. Il est presque temps de passer à l'action, avant cela [un tutorial](https://docs.ray.io/en/master/serve/tutorials/sklearn.html) sur le déploiement d'un modèle sklearn, après l'avoir lu vous serez prêts à implémenter l'API et la déployer sans aucune aide.

**N'utilisez pas le notebook jupyter pour faire l'exercice, un terminal Python et votre IDE feront mieux l'affaire.**

### Découverte de la librairie et implémentation

Pour définir un backend, la première étape est d'implémenter une fonction ou classe qui se chargera de la logique métier une fois la requête réceptionnée. Cette fonction/classe recevra en entrée un objet Flask Request et retournera en sortie un objet serialisable sous forme JSON.

<img src='./images/logo_oc.png' width=15px /> Faites une copie du pipeline joblib dans le répertoire de `ray_serve_api`.

<img src='./images/logo_oc.png' width=15px /> Implémentez le backend de l'API dans le fichier `ray_serve_api/backend.py`. Utilisez une classe pour cela, son constructeur doit prendre en paramètre le chemin du pipeline qui sera chargé par joblib.

<img src='./images/logo_oc.png' width=15px /> Après avoir ouvert une console Python, déclarez un backend en n'oubliant pas de passer en paramètre le chemin du pipeline sérialisé.

<img src='./images/logo_oc.png' width=15px /> Listez l'ensemble des backends actifs depuis la console Python.

<img src='./images/logo_oc.png' width=15px /> Depuis la console Python, déclarez un endpoint associé au backend que vous venez juste de créer. N'oubliez pas de donner une route pour ce point d'accès !

**Attention** : l'autre API que nous avons déployé pour le moment (MLFlow), acceptent par défaut une requête HTTP de type POST pour réceptionner la requête. Ray Serve accepte par défaut une requête HTTP de type GET, regardez dans la [documentation](https://docs.ray.io/en/master/serve/package-ref.html#ray.serve.api.Client.create_endpoint) comment modifier cela.

<img src='./images/logo_oc.png' width=15px /> Listez l'ensemble des endpoints actifs depuis la console Python.

<img src='./images/logo_oc.png' width=15px /> Dans le fichier `dashboard.py` affectez l'adresse du endpoint Ray Serve à la variable `RAY_SERVE_URI` dans la fonction `main`, puis vérifiez que cela fonctionne correctement.

<img src='./images/logo_oc.png' width=15px /> Vous allez maintenant simuler la mise à jour du pipeline ! en entreprise, nous sommes souvent amenés à mettre à jour les modèles à la suite d'amélioration de performances ou encore à l'évolution des tendances présentes dans les données.

Pour simuler cette [mise à jour](https://docs.ray.io/en/master/serve/deployment.html#updating-your-model-over-time), nous allons simplement créer un nouveau backend avec le même pipeline que précédemment (oui, ce n'est pas réaliste !), puis vous **redirigerez 80% du traffic vers le nouveau backend**.

Si vous souhaitez **augmenter le nombre de requêtes** concurrentes que votre API peut traiter, augmentez le nombre de répliques du backend, c'est [très simple](https://docs.ray.io/en/master/serve/advanced.html#scaling-out).

### Conclusion et perspectives d'utilisation de Ray Serve
Ray Serve est une librairie qui se montre très simple d'utilisation pour déployer des services implémentés en Python. Malgré sa simplicité, vous pouvez comprendre qu'elle nous permet de déployer et gérer un ensemble de services sur une infrastructure distribuée comme un cluster Kubernetes. C'est un outil bien intégré dans la communauté scientifique de Python, mais qui à l'inverse des deux outils précédents, n'a fait aucun choix d'infrastructure cloud, c'est volontaire !

N'oubliez pas que Ray propose d'autres librairies, dont une qui devrait beaucoup vous intéresser pour la recherche des hyperparamètres, c'est Ray Tune !

## 5 - Bonus - Déploiement d'un modèle sklearn avec Cortex (30 minutes)

### Pré-requis

**Cortex fonctionne uniquement sur Linux/MacOS**, vous devriez être capable d'utiliser une distribution de Linux comme Ubuntu, à travers une machine virtuelle par exemple. Cependant, si cela vous semble trop compliqué, vous pouvez passer l'exercice si vous le souhaitez ;)

**Cortex repose sur docker pour fonctionner, vous devrez donc l'[installer](https://docs.docker.com/get-docker/)**

### Installation avec Pip

<img src='./images/logo_oc.png' width=15px /> Exécutez la commande suivante depuis votre terminal `pip install cortex`

### Présentation de Cortex

Cortex est une librairie Python permettant de :
- Déployer des modèles (TensorFlow, PyTorch, ONNX, scikit-learn, etc) avec la possibilité de définir des étapes de pré/post-traitement.
- Gérer les APIs une fois déployées (monitoring, mise à jour, logging, A/B testing).
- Contrôler rapidement le nombre d'instances AWS en cas de montée de charge.

L'atout de Cortex est la facilité de déploiement sur le cloud AWS (et bientôt GCP), nous n'utiliserons pas cette fonctionnalité, mais sachez qu'elle est disponible.

De la même façon qu'avec MLflow, notre objectif est d'implémenter et de déployer une API à l'aide de Cortex cette fois-ci.


### Découverte de la librairie et implémentation

<img src='./images/logo_oc.png' width=15px /> Lisez dans un premier temps la [page de documentation générale](https://docs.cortex.dev/)

<img src='./images/logo_oc.png' width=15px /> En parcourant rapidement ce [tutorial](https://docs.cortex.dev/v/0.21/deployments/realtime-api/predictors) vous devriez ensuite être en mesure d'implémenter l'interface de prédiction pour Python (fichier `predictor.py`), puis la configuration de l'API (fichier `cortex.yaml`).

Si nécessaire [ce tutorial](https://github.com/cortexlabs/cortex/tree/0.21/examples/sklearn/iris-classifier) donne des pistes sur l'utilisation de Cortex pour sklearn.

<img src='./images/logo_oc.png' width=15px /> Faites une copie du pipeline joblib dans le répertoire `cortex_api`.

<img src='./images/logo_oc.png' width=15px /> Implémentez la classe `PythonPredictor` dans le fichier `cortex_api/predictor.py`, le constructeur doit charger le pipeline dans un attribut de la classe, puis la fonction `predict` doit retourner une prédiction en utilisant le pipeline chargé.

<img src='./images/logo_oc.png' width=15px /> Implémentez la configuration de l'API dans le fichier `cortex_api/cortex.yaml`, cela permettra de déployer plus tard. Vous n'avez besoin de renseigner que [les champs](https://docs.cortex.dev/v/0.21/deployments/realtime-api/api-configuration) `name`, `kind` et `predictor`.

<img src='./images/logo_oc.png' width=15px /> Vous êtes maintenant prêts pour déployer, ouvrez un terminal dans le répertoire de code de Cortex. Déployez l'API et vérifiez qu'elle fonctionne. Des exemples de commandes à utiliser :

```bash
# deploy the model as a realtime api
cortex deploy

# view the status of the api
cortex get --watch

# stream logs from the api
cortex logs text-generator

# get the api's endpoint
cortex get text-generator
```

<img src='./images/logo_oc.png' width=15px />  Dans le fichier `dashboard.py` affectez l'adresse du serveur de l'API à la variable `CORTEX_URI` dans la fonction `main`, puis vérifiez que cela fonctionne correctement.

### Conclusion et perspectives de Cortex

Nous venons tout juste de déployer le pipeline avec Cortex en local, en réalité nous n'avons pas réellement tiré parti de cet outil car son rôle principal est de faciliter le déploiement dans le cloud. Pour le moment l'outil fonctionne uniquement avec AWS, si vous avez des ressources, le fichier de configuration de l'API `cortex.yaml` contient de nombreux paramètres pour générer l'allocation des ressources, le réseau, le monitoring ou encore la montée en charge automatique.

## Conclusion générale

À travers ce tutorial, vous avez comparé trois outils qui proposent au moins une fonctionnalité de déploiement d'API orientée pour le machine learning :
- MLflow pour déployer en local ou sur les clouds (AWS, Azure) et qui dispose d'autres fonctionnalités comme le suivi d'expérience, la création d'un registre de modèles.
- Ray Tune pour déployer en local ou sur un cluster que vous gérez, propose la création rapide d'un backend (modèle ou logique métier) et d'un point d'accès associé. Vous pourrez gérer la montée en puissance des vos services directement depuis la console Python.
- Cortex pour déployer en local ou sur AWS (bientôt GCP). Cortex se focalise exclusivement sur le déploiement et le cycle de vie du modèle, avec de nombreuses options pour paramétrer l'infrastructure.

À vous d'utiliser la librairie qui semble la plus adaptée à votre projet / entreprise / cloud !